# To Do

* parallelize experimental pipeline to make experiment iterations go faster on jakku
* experiment with different clustering approaches
    * different values for K in KMeans
    * DBScan or some other clustering approach
    * stratify data across activity classes to match labeled data
* experiment with uncertainty sampling instead of random sampling of personal data
* investigate clusters
    * How big are the clusters? Is it just a matter of size of the cluster?
    * are the clusters fairly homogenous/heterogenous with respect to
        * activity
        * user data (is a user's data mostly within one cluster, or many?)
* investigate why this method works great for some people, but not others
* What does this mean for designing human-in-the-loop systems?

# Experiment 1

Comparing Universal + Personal data with varying amounts of personal data to 1 cluster of universal + personal data
Note : This experiment was only conducted on the WISDM v1.1 dataset and should also be conducted on the WISDM v2.0 dataset.

### Result : 
On average, the nearest cluster + personal data did about the same as the universal + personal data, but there were far fewer really bad scenarios especially with little labeled data. Also, for some individuals, this approach consistently proved to perform best.

### Follow-up Question #1: What is different about the moments when this approach works and when it doesn't work?

### Follow-up Question #2: Can we determine early in the use of the AR technology whether the personal, universal + personal, or cluster + personal approach will work best?

# Experiment 2
Using an uncertainty sampling approach to intelligently select data from available personal training set.